In [1]:
import pathlib
import warnings

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.utils import parallel_backend
from sklearn.exceptions import ConvergenceWarning
from joblib import load

import sys

sys.path.append("../utils")
from split_utils import get_features_data
from train_utils import get_dataset, get_X_y_data

In [2]:
# load labeled data
labeled_data_path = pathlib.Path("../0.download_data/data/labeled_data.csv.gz")
labeled_data = get_features_data(labeled_data_path)

# preview labeled data
print(labeled_data.shape)
labeled_data.head(5)

(2862, 1458)


,Mitocheck_Phenotypic_Class,Cell_UUID,Location_Center_X,Location_Center_Y,Metadata_Plate,Metadata_Well,Metadata_Frame,Metadata_Site,Metadata_Plate_Map_Name,Metadata_DNA,...,DP__efficientnet_1270,DP__efficientnet_1271,DP__efficientnet_1272,DP__efficientnet_1273,DP__efficientnet_1274,DP__efficientnet_1275,DP__efficientnet_1276,DP__efficientnet_1277,DP__efficientnet_1278,DP__efficientnet_1279
0,Large,21da27ab-873a-41f4-ab98-49170cae9a2d,397,618,LT0010_27,173,83,1,LT0010_27_173,LT0010_27/LT0010_27_173_83.tif,...,1.526493,-0.388909,-0.715202,-0.939279,-0.077689,1.965509,18.685819,0.061676,2.641369,-0.086854
1,Large,82f7949b-4ea2-45c8-8dd9-7854caf49077,359,584,LT0010_27,173,83,1,LT0010_27_173,LT0010_27/LT0010_27_173_83.tif,...,-0.482883,-1.354858,-0.856680,-0.934949,0.725091,2.255450,-0.565433,1.628086,-0.605625,-0.748135
2,Large,cec7234f-fe35-4411-aded-f8112bb31219,383,685,LT0010_27,173,83,1,LT0010_27_173,LT0010_27/LT0010_27_173_83.tif,...,0.888706,1.350431,-0.648841,0.264205,0.131341,0.678315,0.171044,0.342206,-0.581597,0.505556
3,Large,43d9e7c9-c9ec-45ce-8820-048bfb896989,932,532,LT0013_38,42,75,1,LT0013_38_42,LT0013_38/LT0013_38_42_75.tif,...,-1.001625,-0.801021,-0.586539,0.076197,0.599191,1.742090,0.365520,0.643759,-1.906097,1.019370
4,Large,63ce6652-338e-4afd-9c77-dbc0e903bf92,477,130,LT0013_38,42,75,1,LT0013_38_42,LT0013_38/LT0013_38_42_75.tif,...,0.950706,-0.811825,-0.522427,-1.402842,-0.289940,2.661250,0.126978,-0.824945,-0.494285,1.763332


In [3]:
# see number of images to
num_images = labeled_data["Metadata_DNA"].unique().shape[0]
print(f"There are {num_images} images to perform LOIO evaluation on per model.")


There are 270 images to perform LOIO evaluation on per model.


In [4]:
# directory to load the models from
models_dir = pathlib.Path("../2.train_model/models/")

# use a list to keep track of LOIO probabilities in tidy long format for each model combination
compiled_LOIO_wide_data = []

count = 0

# iterate through each model (final model, shuffled baseline model, etc)
# sorted so final models are loaded before shuffled_baseline
for model_path in sorted(models_dir.iterdir()):
    # only perform LOIO with hyper params from final models
    if "shuffled" in model_path.name:
        continue

    # load the model
    model = load(model_path)
    # determine feature type from model file name
    feature_type = model_path.name.split("__")[1].replace(".joblib", "")

    print(
        f"Performing LOIO for feature type {feature_type} with parameters C: {model.C}, l1_ratio: {model.l1_ratio}"
    )

    # iterate through image paths
    for image_path in labeled_data["Metadata_DNA"].unique():
        # get training and testing cells from image path
        # every cell from the image path is for testing, the rest are for training
        train_cells = labeled_data.loc[labeled_data["Metadata_DNA"] != image_path]
        test_cells = labeled_data.loc[labeled_data["Metadata_DNA"] == image_path]

        # get X, y from training and testing cells
        X_train, y_train = get_X_y_data(train_cells, feature_type)
        X_test, y_test = get_X_y_data(test_cells, feature_type)

        # capture convergence warning from sklearn
        # this warning does not affect the model but takes up lots of space in the output
        with parallel_backend("multiprocessing"):
            with warnings.catch_warnings():
                warnings.filterwarnings(
                    "ignore", category=ConvergenceWarning, module="sklearn"
                )

                # fit a logisitc regression model on the training X, y
                LOIO_model = LogisticRegression(
                    penalty="elasticnet",
                    solver="saga",
                    max_iter=100,
                    n_jobs=-1,
                    random_state=0,
                    C=model.C,
                    l1_ratio=model.l1_ratio,
                ).fit(X_train, y_train)

        # create metadata dataframe for test cells with model parameters
        metadata_dataframe = pd.concat(
            [
                test_cells["Cell_UUID"],
                test_cells["Metadata_DNA"],
                test_cells["Mitocheck_Phenotypic_Class"],
            ],
            axis=1,
        ).reset_index(drop=True)
        metadata_dataframe["Model_Feature_Type"] = feature_type
        metadata_dataframe["Model_C"] = model.C
        metadata_dataframe["Model_l1_ratio"] = model.l1_ratio

        # predict probabilities for test cells and make these probabilities into a dataframe
        probas = LOIO_model.predict_proba(X_test)
        probas_dataframe = pd.DataFrame(probas, columns=model.classes_)

        # combine metadata and probabilities dataframes for test cells to create wide data
        test_cells_wide_data = pd.concat([metadata_dataframe, probas_dataframe], axis=1)

        # add tidy long data to compiled data
        compiled_LOIO_wide_data.append(test_cells_wide_data)

        # DELETE THE REST OF THE LINES IN THIS NOTEBOOK BEFORE FINAL RUN
        score = LOIO_model.score(X_test, y_test)
        print(
            f"Leaving out image: {image_path}; number of cells: {test_cells.shape[0]}, score: {score}"
        )

        count += 1
        if count % 5 == 0:
            break


Performing LOIO for feature type CP with parameters C: 1.0, l1_ratio: 0.30000000000000004
Leaving out image: LT0010_27/LT0010_27_173_83.tif; number of cells: 24, score: 0.7083333333333334
Leaving out image: LT0013_38/LT0013_38_42_75.tif; number of cells: 15, score: 0.6
Leaving out image: LT0013_38/LT0013_38_42_95.tif; number of cells: 22, score: 0.8636363636363636
Leaving out image: LT0017_19/LT0017_19_365_65.tif; number of cells: 9, score: 0.7777777777777778
Leaving out image: LT0019_06/LT0019_06_140_70.tif; number of cells: 29, score: 0.7586206896551724
Performing LOIO for feature type CP_and_DP with parameters C: 0.1, l1_ratio: 0.30000000000000004
Leaving out image: LT0010_27/LT0010_27_173_83.tif; number of cells: 24, score: 0.8333333333333334
Leaving out image: LT0013_38/LT0013_38_42_75.tif; number of cells: 15, score: 0.7333333333333333
Leaving out image: LT0013_38/LT0013_38_42_95.tif; number of cells: 22, score: 0.9090909090909091
Leaving out image: LT0017_19/LT0017_19_365_65.tif

In [5]:
# compile list of wide data into one dataframe
compiled_LOIO_wide_data = pd.concat(compiled_LOIO_wide_data).reset_index(drop=True)

# convert wide data to tidy long data and sort by Cell_UUID, Model_Feature_Type, and Model_Phenotypic_Class for pretty formatting
compiled_LOIO_tidy_long_data = (
    pd.melt(
        compiled_LOIO_wide_data,
        id_vars=metadata_dataframe.columns,
        value_vars=probas_dataframe.columns,
        var_name="Model_Phenotypic_Class",
        value_name="Predicted_Probability",
    )
    .sort_values(["Model_Feature_Type", "Cell_UUID", "Model_Phenotypic_Class"])
    .reset_index(drop=True)
)

# specify results directory
LOIO_probas_dir = pathlib.Path("evaluations/LOIO_probas/")
LOIO_probas_dir.mkdir(parents=True, exist_ok=True)

# define save path
compiled_LOIO_save_path = pathlib.Path(
    f"{LOIO_probas_dir}/compiled_LOIO_probabilites.tsv"
)

# save data as tsv
compiled_LOIO_tidy_long_data.to_csv(compiled_LOIO_save_path, sep="\t")

# preview tidy long data
compiled_LOIO_tidy_long_data


,Cell_UUID,Metadata_DNA,Mitocheck_Phenotypic_Class,Model_Feature_Type,Model_C,Model_l1_ratio,Model_Phenotypic_Class,Predicted_Probability
0,009a4190-7583-4821-9a17-737f0485d252,LT0010_27/LT0010_27_173_83.tif,Elongated,CP,1.0,0.3,ADCCM,0.000290
1,009a4190-7583-4821-9a17-737f0485d252,LT0010_27/LT0010_27_173_83.tif,Elongated,CP,1.0,0.3,Anaphase,0.000040
2,009a4190-7583-4821-9a17-737f0485d252,LT0010_27/LT0010_27_173_83.tif,Elongated,CP,1.0,0.3,Apoptosis,0.000022
3,009a4190-7583-4821-9a17-737f0485d252,LT0010_27/LT0010_27_173_83.tif,Elongated,CP,1.0,0.3,Binuclear,0.000015
4,009a4190-7583-4821-9a17-737f0485d252,LT0010_27/LT0010_27_173_83.tif,Elongated,CP,1.0,0.3,Elongated,0.991291
...,...,...,...,...,...,...,...,...
4450,fd1e0290-f17d-492e-bd8a-6af449e6b417,LT0019_06/LT0019_06_140_70.tif,Apoptosis,DP,1.0,1.0,MetaphaseAlignment,0.001394
4451,fd1e0290-f17d-492e-bd8a-6af449e6b417,LT0019_06/LT0019_06_140_70.tif,Apoptosis,DP,1.0,1.0,OutOfFocus,0.000410
4452,fd1e0290-f17d-492e-bd8a-6af449e6b417,LT0019_06/LT0019_06_140_70.tif,Apoptosis,DP,1.0,1.0,Polylobed,0.305711
4453,fd1e0290-f17d-492e-bd8a-6af449e6b417,LT0019_06/LT0019_06_140_70.tif,Apoptosis,DP,1.0,1.0,Prometaphase,0.024472
